In [9]:
import pandas as pd
from pandas import DataFrame,merge,concat,read_csv
import numpy as np
from requests import get
import crisjf_functions as cf
import matplotlib.pyplot as plt
%matplotlib inline


In [6]:
#Import data

#Import export and RCA
df = DataFrame.from_csv("i_year_origin_hs02_4.tsv", sep="\t")

df = df.replace(np.nan, 0)

#Products
prods = DataFrame.from_csv("products_hs_02.tsv", sep="\t",header=0)

#Print heading
print(df.head())
print(prods.head())

# df.to_csv('RCAdata.csv')
# prods.to_csv('HS02data.csv')

wt, pr, co = cf.trade_data('hs02')

   year origin  hs02  export_val  import_val  export_rca  import_rca
1                                                                   
2  2003    ago   101        0.00     18430.0    0.000000    0.016827
3  2003    ago   102    17821.54   2566670.0    0.004657    1.192530
4  2003    ago   104     6711.52     49352.0    0.007064    0.092357
5  2003    ago   105        0.00    160058.0    0.000000    0.264245
6  2003    ago   106     6458.00     94913.0    0.011895    0.310840
            hs02                                               name
id                                                                 
0101          01                                       Live animals
010101      0101             Live horses, asses, mules and hinnies.
01010110  010110  Live horses/asses/mules/hinnies: pure-bred bre...
01010190  010190  Live horses/asses/mules/hinnies other than pur...
010102      0102                               Live bovine animals.


In [11]:
# Binarize RCA values
wt['export_rca_bin'] = wt['export_rca'] >= 1
# Select rows with high RCA and group
wt = wt.loc[wt['export_rca_bin'] == True]
wtg = wt.groupby(['ccode', "pcode"])
keys_select = wtg.groups.keys()[:10]
keys_select

[('brb', 3814),
 ('bra', 2924),
 ('aut', 7311),
 ('che', 3909),
 ('dnk', 5701),
 ('aut', 2932),
 ('and', 7608),
 ('geo', 8449),
 ('com', 6306),
 ('cym', 7208)]

In [13]:
def complete(key):
    """
    Completes entries in the groups so that they have nice structure
    """
    gr = wtg.get_group(key)
    new_index = pd.Index(range(2003,2016), name="year")
    gr.set_index('year')
    gr.set_index("year").reindex(new_index)
    df = gr.set_index("year").reindex(new_index).reset_index()
    c = gr['ccode'].value_counts().idxmax()
    df['ccode'] = c
    p = gr['pcode'].value_counts().idxmax()
    df['pcode'] = p
    df['export_rca_bin'] = np.logical_not([np.isnan(i) for i in df['export_rca_bin'].values]).astype('int')
    df['diff'] = df['export_rca_bin'].diff()
#     gr = df
    return df

def summarize(df):
    """
    From the output of the function above, make a summary table with one row for each event of RCA going above threshold
    """
    events = df.loc[df['diff'] != 0, ('year','ccode','pcode','export_rca_bin')].reset_index()
    # events['diff2'] = events['export_rca_bin'].diff()
    ev_sum = events[events['export_rca_bin'] == 1]
    idxs = np.array(ev_sum.index)
    year_up = np.array(events.ix[idxs]['year'])
    year_down = np.array(events.ix[idxs+1]['year'])
    ev_sum = DataFrame(data={'year_up': year_up, 'year_down': year_down, 'ccode': df['ccode'], 'pcode' : df['pcode'], 'diff' : year_down-year_up}, index = range(len(year_up)))
    ev_sum['end_of_sample'] = ev_sum['year_down'] == 2015
    ev_sum['start_of_sample'] = ev_sum['year_up'] == 2003
    return ev_sum

In [14]:
#To make calculations faster, do them with small data
keys_select = wtg.groups.keys()[:10]
data = pd.DataFrame([])

#Append all summary tables
for key in keys_select:
    data = data.append(pd.DataFrame(summarize(complete(key))), ignore_index=True)

data

,ccode,diff,pcode,year_down,year_up,end_of_sample,start_of_sample
0,brb,3,3814,2010,2007,False,False
1,brb,1,3814,2012,2011,False,False
2,brb,2,3814,2015,2013,True,False
3,bra,1,2924,2009,2008,False,False
4,aut,12,7311,2015,2003,True,True
5,che,7,3909,2010,2003,False,True
6,dnk,1,5701,2015,2014,True,False
7,aut,1,2932,2008,2007,False,False
8,aut,1,2932,2010,2009,False,False
9,aut,1,2932,2012,2011,False,False


In [ ]:


# wt1 = wt.loc[wt['export_rca'] >1]
# x = wt1.loc[wt['year'] == 2014]
# x['pcode'] = x['pcode'].apply(str)

# co_contcode = {v: k for k, v in dict(enumerate(np.unique(x['ccode']))).iteritems()}
# pr_contcode = {v: k for k, v in dict(enumerate(np.unique(x['pcode']))).iteritems()}

# x = x.replace({"ccode": co_contcode})
# x = x.replace({"pcode": pr_contcode})
# # x
# x = np.asarray(x, dtype=float)
# x = np.array(x[:,(1,2)], dtype = int)

In [ ]:
# ind = [(row[0], row[1]) for row in x]

# RCA = np.zeros((max(x[:,0])+1,max(x[:,1])+1),dtype=bool)

# for i in ind:
#     RCA[i] = True


In [ ]:
# plt.figure(figsize=(10,2))
# plt.imshow(RCA,cmap=plt.cm.Greys)
# plt.show()

In [ ]:
# plt.figure(figsize=(25,2))
# order = np.argsort([sum(country) for country in RCA])
# RCAco = RCA[order]
# plt.imshow(RCAco,cmap=plt.cm.Greys)
# order2 = np.argsort([sum(prod) for prod in np.transpose(RCAco)])
# RCAcopo = np.transpose(np.transpose(RCAco)[order2])
# plt.imshow(RCAcopo,cmap=plt.cm.Greys)
# plt.show()

In [ ]:
# wt1 = wt.loc[wt['export_rca'] >1]
# x = wt1.loc[wt['year'] == 2014]
# x['pcode'] = x['pcode'].apply(str)

# co_contcode = {v: k for k, v in dict(enumerate(np.unique(x['ccode']))).iteritems()}
# pr_contcode = {v: k for k, v in dict(enumerate(np.unique(x['pcode']))).iteritems()}

# x = x.replace({"ccode": co_contcode})
# x = x.replace({"pcode": pr_contcode})
# # x
# x = np.asarray(x, dtype=float)
# x = np.array(x[:,(1,2)], dtype = int)

In [ ]:
# arg = wt.loc[(wt['ccode'] == 'arg')]

# n = len(arg)


# arg['rca_perc'] = arg['export_rca']
# # grouped = arg.groupby('year')
# # grouped.agg({'year': arg['rca_perc'].rank(method = 'max')})
# arg['rca_perc'] = arg['rca_perc'].rank(method = 'max')/n
# arg
